In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy
import scipy.io as sio
import torch
import torch.nn.functional as F
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from scipy.spatial.distance import pdist, squareform
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ExponentialLR, LinearLR

from splice import splice_model
from splice.base import *
from splice.loadCellsFile import *
from splice.utilities import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Harris_Lab\miniconda3\envs\splice\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-26 08:17:06,096	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-26 08:17:06,706	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:
filepath = r"C:\Users\Harris_Lab\Projects\SPLICE\data\uberphys\split_rates.npz"
split_rates = np.load(filepath)
train_ads = torch.Tensor(split_rates['train_ads']).to(device)
train_m1 = torch.Tensor(split_rates['train_m1']).to(device)
test_ads = torch.Tensor(split_rates['test_ads']).to(device)
test_m1 = torch.Tensor(split_rates['test_m1']).to(device)

In [28]:
model = splice_model(
    n_a = train_ads.shape[1],
    n_b = train_m1.shape[1],
    n_shared = 2,
    n_priv_a = 2,
    n_priv_b = 2,
    layers_enc = [20,20],
    layers_dec = [20],
    layers_msr = [20,20],
).to(device)

In [29]:
model.fit(train_ads, train_m1, test_ads, test_m1, disent_start=5000, c_disent=0.5, epochs=50000, weight_decay=1e-4)

Epoch 0 	 source loss: 41.0616 | 40.3284 	 target loss: 257.0844 | 264.1692 	 disent loss: 1.0000 | 0.0823 	 msr loss: 0.0000 | 4.8834
Epoch 500 	 source loss: 18.0103 | 18.1820 	 target loss: 67.5321 | 79.5731 	 disent loss: 1.0000 | 0.4374 	 msr loss: 0.0000 | 13.1859
Epoch 1000 	 source loss: 16.6460 | 17.2174 	 target loss: 58.6734 | 76.2645 	 disent loss: 1.0000 | 1.7294 	 msr loss: 0.0000 | 12.8936
Epoch 1500 	 source loss: 16.0966 | 16.8654 	 target loss: 47.3271 | 63.0033 	 disent loss: 1.0000 | 4.7268 	 msr loss: 0.0000 | 19.6811
Epoch 2000 	 source loss: 15.8241 | 16.7150 	 target loss: 45.3304 | 55.4812 	 disent loss: 1.0000 | 7.1361 	 msr loss: 0.0000 | 27.9648
Epoch 2500 	 source loss: 15.6301 | 16.6984 	 target loss: 43.5824 | 51.3354 	 disent loss: 1.0000 | 10.1719 	 msr loss: 0.0000 | 40.0016
Epoch 3000 	 source loss: 15.4599 | 16.7443 	 target loss: 42.3436 | 49.0680 	 disent loss: 1.0000 | 14.1910 	 msr loss: 0.0000 | 53.7456
Epoch 3500 	 source loss: 15.3343 | 16.738

KeyboardInterrupt: 